# Локальное использование LLM. Практика

## Введение

**Цель занятия:** 
Перейти от теоретического понимания архитектур к практическому развертыванию современных LLM на локальном «железе». Мы исследуем границы возможностей видеокарты потребительского уровня (RTX 4070) и научимся выбирать оптимальный стек под конкретные задачи.

### Что мы делаем в рамках практики:
*   **Сравниваем точность и потребление:** На примере моделей семейства **Qwen3 (4B и 8B)** мы увидим разницу между «сырым» весом (BF16) и современным квантованием (NF4).
*   **Исследуем движки инференса:** Мы пройдем путь от гибкого, но тяжелого `Transformers`, через оптимизированный `Unsloth`, до высокопроизводительного `Llama.cpp` и готового к продакшену `Ollama`.
*   **Изучаем паттерны:** Тестируем работу моделей в режиме «рассуждения» (**Thinking Mode**) и смотрим, как это влияет на задержку и расчет скорости генерации (TPS).
*   **Анализируем гибридные режимы:** Увидим на практике, что происходит со скоростью, когда модель перестает влезать в видеопамять и задействует ресурсы оперативной памяти.

## Импорты и Утилиты

In [1]:
from unsloth import FastLanguageModel

import torch
import gc
import time
import os
from IPython.display import display, Markdown
from tabulate import tabulate

# Библиотеки для разных движков
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import openai

# Глобальный список для хранения результатов всех тестов
results = []

def clean_memory():
    """Принудительная очистка VRAM и Garbage Collector"""
    # Удаляем переменные моделей, если они остались в глобальной области
    for var in ['model', 'tokenizer', 'llm', 'llm_hybrid']:
        if var in globals():
            del globals()[var]
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()
    print("🧹 Память очищена")

def print_memory():
    """Вывод занятой памяти"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1024**3
        reserved = torch.cuda.memory_reserved(0) / 1024**3
        print(f"📊 VRAM: Занято {allocated:.2f} ГБ | Зарезервировано {reserved:.2f} ГБ")
    else:
        print("❌ GPU не найден")

print_memory()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
📊 VRAM: Занято 0.00 ГБ | Зарезервировано 0.00 ГБ


## Математика памяти (Теория)

In [3]:
def estimate_vram(params_b, quant_type="fp16", context=8192):
    # Таблица размеров типов данных
    bytes_per_param = {
        "fp16": 2,      # 16 bit = 2 байта (BF16/FP16)
        "int8": 1,      # 8 bit = 1 байт
        "int4": 0.55    # 4 bit = ~0.55 байта (с учетом метаданных квантования)
    }
    
    bpp = bytes_per_param.get(quant_type, 2)
    
    # Вес весов модели
    model_size = params_b * bpp
    
    # KV-Cache (память под контекст)
    # Формула для Qwen3 (GQA): ~0.5 МБ на 1к токенов * слои/головы (упрощенно)
    # Для 8B модели при 8k контекста это около 1-1.5 ГБ в FP16
    kv_cache_gb = (context / 1000) * 0.15 
    
    total = model_size + kv_cache_gb
    return f"🤖 Qwen3-{params_b}B ({quant_type}): Вес ~{model_size:.1f} GB + Кэш ~{kv_cache_gb:.1f} GB = ИТОГО: {total:.2f} GB"

print("--- 🛑 Железо: RTX 4070 (Лимит 8 GB) ---")

# 1. Оригинальная модель (как на HuggingFace)
print(estimate_vram(8, "fp16")) 
# Вывод: ~17 ГБ. Сразу OOM. Студенты видят, почему нельзя просто сделать .from_pretrained()

# 2. То, что мы будем делать (NF4 / Unsloth)
print(estimate_vram(8, "int4")) 
# Вывод: ~5.5 ГБ. Идеально влезает, остается ~2.5 ГБ под систему и длинный контекст.

# 3. Мечта (32B модель)
print(estimate_vram(32, "int4")) 
# Вывод: ~18 ГБ. На ноутбуке без шансов (только через медленный RAM offload).

--- 🛑 Железо: RTX 4070 (Лимит 8 GB) ---
🤖 Qwen3-8B (fp16): Вес ~16.0 GB + Кэш ~1.2 GB = ИТОГО: 17.23 GB
🤖 Qwen3-8B (int4): Вес ~4.4 GB + Кэш ~1.2 GB = ИТОГО: 5.63 GB
🤖 Qwen3-32B (int4): Вес ~17.6 GB + Кэш ~1.2 GB = ИТОГО: 18.83 GB


## Qwen3-4B в полном весе (BF16)

In [4]:
clean_memory()

model_id = "Qwen/Qwen3-4B"
print(f"🏋️ Загрузка {model_id} в полном весе (BF16)...")

# device_map="auto" спасет от краша, если памяти чуть не хватит
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.bfloat16, 
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

print_memory()

# Тест генерации
input_text = "Explain the concept of entropy in simple terms."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

start = time.time()
output = model.generate(**inputs, max_new_tokens=100)
end = time.time()

# Считаем скорость
new_tokens = len(output[0]) - len(inputs.input_ids[0])
tps = new_tokens / (end - start)

print(f"⏱ Скорость (BF16): {tps:.2f} токенов/сек")
results.append(["Qwen3-4B (BF16)", f"{torch.cuda.memory_allocated(0)/1024**3:.2f} GB", f"{tps:.2f} t/s"])

del model, tokenizer
clean_memory()

🧹 Память очищена
🏋️ Загрузка Qwen/Qwen3-4B в полном весе (BF16)...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[accelerate.big_modeling|WARNING]Some parameters are on the meta device because they were offloaded to the cpu.


📊 VRAM: Занято 6.37 ГБ | Зарезервировано 6.39 ГБ
⏱ Скорость (BF16): 3.00 токенов/сек
🧹 Память очищена


## Qwen3-4B квантованная (NF4)

In [5]:
clean_memory()

print(f"🤏 Загрузка {model_id} в квантовании NF4...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

print_memory() # Должно быть около 3 ГБ

inputs = tokenizer("Explain the concept of entropy in simple terms.", return_tensors="pt").to("cuda")

start = time.time()
output = model.generate(**inputs, max_new_tokens=100)
end = time.time()

new_tokens = len(output[0]) - len(inputs.input_ids[0])
tps = new_tokens / (end - start)

print(f"⏱ Скорость (NF4): {tps:.2f} токенов/сек")
results.append(["Qwen3-4B (NF4)", f"{torch.cuda.memory_allocated(0)/1024**3:.2f} GB", f"{tps:.2f} t/s"])

del model, tokenizer
clean_memory()

🧹 Память очищена
🤏 Загрузка Qwen/Qwen3-4B в квантовании NF4...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

📊 VRAM: Занято 2.65 ГБ | Зарезервировано 3.20 ГБ
⏱ Скорость (NF4): 24.51 токенов/сек
🧹 Память очищена


## Qwen3-8B (Transformers + Thinking Mode)

In [6]:
clean_memory()

model_id_8b = "Qwen/Qwen3-8B"
print(f"🧠 Загрузка {model_id_8b} (HF NF4) с Thinking Mode...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id_8b,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id_8b)

# Подготовка промпта для Thinking Mode
prompt = "How many 'r's are in the word 'Strawberry'?"
messages = [{"role": "user", "content": prompt}]

# Включаем enable_thinking=True
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True 
)
model_inputs = tokenizer([text], return_tensors="pt").to("cuda")

start = time.time()
generated_ids = model.generate(**model_inputs, max_new_tokens=512)
end = time.time()

output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)

print("\n--- RAW OUTPUT ---")
print(output_text[:500] + "...") 

new_tokens = len(generated_ids[0]) - len(model_inputs.input_ids[0])
tps = new_tokens / (end - start)

print(f"\n⏱ Скорость (HF 8B): {tps:.2f} токенов/сек")
results.append(["Qwen3-8B (HF NF4)", f"{torch.cuda.memory_allocated(0)/1024**3:.2f} GB", f"{tps:.2f} t/s"])

del model, tokenizer
clean_memory()

🧹 Память очищена
🧠 Загрузка Qwen/Qwen3-8B (HF NF4) с Thinking Mode...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]


--- RAW OUTPUT ---
<|im_start|>user
How many 'r's are in the word 'Strawberry'?<|im_end|>
<|im_start|>assistant
<think>
Okay, let's see. The user is asking how many 'r's are in the word 'Strawberry'. Hmm, first I need to make sure I spell the word correctly. Let me write it out: S-T-R-A-W-B-E-R-R-Y. Wait, let me check again. S-T-R-A-W-B-E-R-R-Y. Yes, that's right. Now, I need to count the number of times the letter 'r' appears.

Starting from the beginning: S is the first letter, then T, then R. So that's one 'r'....

⏱ Скорость (HF 8B): 26.64 токенов/сек
🧹 Память очищена


## Qwen3-8B через Unsloth (Сравнение скорости)

In [7]:
clean_memory()

unsloth_model_id = "unsloth/Qwen3-8B-bnb-4bit" 

print(f"🐇 Unsloth: Загрузка {unsloth_model_id}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=unsloth_model_id,
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

inputs = tokenizer(["How many 'r's are in the word 'Strawberry'?"], return_tensors="pt").to("cuda")

# Прогрев
_ = model.generate(**inputs, max_new_tokens=1)

start = time.time()
outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
end = time.time()

tps = 512 / (end - start) # Примерный подсчет
print(f"⏱ Скорость (Unsloth 8B): {tps:.2f} токенов/сек")
results.append(["Qwen3-8B (Unsloth)", f"{torch.cuda.memory_allocated(0)/1024**3:.2f} GB", f"{tps:.2f} t/s"])

del model, tokenizer
clean_memory()

🧹 Память очищена
🐇 Unsloth: Загрузка unsloth/Qwen3-8B-bnb-4bit...
==((====))==  Unsloth 2026.1.4: Fast Qwen3 patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 4070 Laptop GPU. Num GPUs = 1. Max memory: 7.633 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
⏱ Скорость (Unsloth 8B): 30.68 токенов/сек
🧹 Память очищена


## Llama.cpp (GGUF & Hybrid Inference)

In [8]:
# Установка Llama-cpp-python с поддержкой CUDA
# ВАЖНО: Без CMAKE_ARGS она встанет как CPU-only библиотека.
# CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python

In [9]:
clean_memory()

# 1. Скачиваем GGUF файл в папку ./models
print("📥 Скачивание GGUF...")

model_path = hf_hub_download(
    repo_id="unsloth/Qwen3-8B-GGUF", 
    filename="Qwen3-8B-Q4_K_M.gguf",
    local_dir="./models"
)


print(f"Файл готов: {model_path}")

# 2. Тест Full GPU
print("\n--- Llama.cpp: Full GPU ---")
llm = Llama(
    model_path=model_path,
    n_gpu_layers=-1, 
    n_ctx=4096,
    verbose=False
)

start = time.time()
output = llm.create_chat_completion(
    messages=[{"role": "user", "content": "Explain rust ownership."}],
    max_tokens=256
)
end = time.time()

new_tokens = output['usage']['completion_tokens']  # pyright: ignore[reportIndexIssue]
tps = new_tokens / (end - start)
print(f"Ответ (GPU): {output['choices'][0]['message']['content'][:50]}...")  # pyright: ignore[reportIndexIssue, reportOptionalSubscript]
print(f"⏱ Скорость: {tps:.2f} t/s")

# Для GGUF сложно точно замерить VRAM из Python, ставим примерное значение или 0
results.append(["Qwen-8B (Llama.cpp GPU)", "~5.5 GB", f"{tps:.2f} t/s"])

del llm
clean_memory()

# 3. Тест Hybrid (CPU Offload)
print("\n--- Llama.cpp: Hybrid (CPU+GPU) ---")
llm_hybrid = Llama(
    model_path=model_path,
    n_gpu_layers=10, # Мало слоев в GPU
    n_ctx=4096,
    verbose=False
)

start = time.time()
output = llm_hybrid.create_chat_completion(
    messages=[{"role": "user", "content": "Explain python GIL."}],
    max_tokens=256
)
end = time.time()

new_tokens = output['usage']['completion_tokens']  # pyright: ignore[reportIndexIssue]
tps = new_tokens / (end - start)
print(f"Ответ (Hybrid): {output['choices'][0]['message']['content'][:50]}...")  # pyright: ignore[reportOptionalSubscript, reportIndexIssue]
print(f"⏱ Скорость: {tps:.2f} t/s")
results.append(["Qwen-8B (Hybrid CPU+GPU)", "~2.0 GB VRAM", f"{tps:.2f} t/s"])

del llm_hybrid
clean_memory()

🧹 Память очищена
📥 Скачивание GGUF...
Файл готов: models/Qwen3-8B-Q4_K_M.gguf

--- Llama.cpp: Full GPU ---


llama_context: n_ctx_per_seq (4096) < n_ctx_train (40960) -- the full capacity of the model will not be utilized


Ответ (GPU): <think>
Okay, I need to explain Rust ownership. Le...
⏱ Скорость: 41.31 t/s
🧹 Память очищена

--- Llama.cpp: Hybrid (CPU+GPU) ---


llama_context: n_ctx_per_seq (4096) < n_ctx_train (40960) -- the full capacity of the model will not be utilized


Ответ (Hybrid): <think>
Okay, the user is asking about the Python ...
⏱ Скорость: 5.48 t/s
🧹 Память очищена


## Ollama + OpenAI API

In [10]:
import openai
import time

print("🌐 Ollama (Docker): Запуск со стримингом...")

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama" 
)

try:
    start_time = time.time()
    
    # 1. Включаем stream=True
    # 2. Убираем max_tokens (пусть говорит сколько нужно)
    stream = client.chat.completions.create(
        model="qwen3:8b", 
        messages=[
            {"role": "user", "content": "Write a short poem about Linux."}
        ],
        stream=True,  
        temperature=0.7
    )
    
    print("Ответ модели:")
    collected_messages = []
    
    # Читаем поток по кусочкам
    for chunk in stream:
        chunk_message = chunk.choices[0].delta.content
        if chunk_message:
            print(chunk_message, end="", flush=True) # Печатаем сразу
            collected_messages.append(chunk_message)
            
    end_time = time.time()
    
    # Считаем статистику постфактум
    full_text = "".join(collected_messages)
    token_estimate = len(full_text.split()) * 1.3
    duration = end_time - start_time
    tps = token_estimate / duration
    
    print(f"\n\n⚡ Скорость генерации: ~{tps:.2f} токенов/сек")
    
    # Сохраняем в таблицу
    if 'results' in globals():
        results.append(["Qwen-8B (Ollama)", "Docker / Streaming", f"~{tps:.2f} t/s"])

except Exception as e:
    print(f"\n❌ Ошибка: {e}")

🌐 Ollama (Docker): Запуск со стримингом...
Ответ модели:
**Ode to Linux**  

Core of code, a community's embrace,  
Freedom's call, security's grace.  
A kernel's heart, where commands align,  
In circuits deep, it hums with might.  

From servers tall to desktop's light,  
It bends and grows, a versatile sprite.  
In terminals deep, the power's near,  
A world of tools, both old and new.  

Open-source roots, a shared design,  
Where every line is a collective sign.  
Contributors' hands, both near and far,  
Forge a future that's evermore star.  

So here's to Linux, steadfast and true,  
A digital realm where freedom's hue.  
In every byte, it stands anew.

⚡ Скорость генерации: ~6.15 токенов/сек


## Итоговая таблица

In [11]:
print("### 🏆 Результаты бенчмарка (RTX 4070 Laptop)")
print(tabulate(results, headers=["Модель / Метод", "VRAM", "Скорость"], tablefmt="github"))

### 🏆 Результаты бенчмарка (RTX 4070 Laptop)
| Модель / Метод           | VRAM               | Скорость   |
|--------------------------|--------------------|------------|
| Qwen3-4B (BF16)          | 6.38 GB            | 3.00 t/s   |
| Qwen3-4B (NF4)           | 2.65 GB            | 24.51 t/s  |
| Qwen3-8B (HF NF4)        | 6.02 GB            | 26.64 t/s  |
| Qwen3-8B (Unsloth)       | 5.82 GB            | 30.68 t/s  |
| Qwen-8B (Llama.cpp GPU)  | ~5.5 GB            | 41.31 t/s  |
| Qwen-8B (Hybrid CPU+GPU) | ~2.0 GB VRAM       | 5.48 t/s   |
| Qwen-8B (Ollama)         | Docker / Streaming | ~6.15 t/s  |


## Выводы по итогам экспериментов

Проведенные тесты демонстрируют ключевые принципы локального инференса LLM в 2026 году:

1.  **Квантование — это стандарт.** Использование 4-битных форматов (NF4, GGUF) позволяет запускать модели среднего размера (8B+) на игровых ноутбуках, высвобождая до 60–70% видеопамяти при минимальной потере качества. Без квантования запуск SOTA-моделей остается привилегией серверных систем.
2.  **Экосистема и оптимизация:** 
    *   **Unsloth** подтверждает свою эффективность для R&D, обеспечивая прирост скорости относительно стандартных реализаций за счет оптимизированных ядер Triton.
    *   **Llama.cpp** остается эталоном производительности для инференса, максимально эффективно утилизируя ресурсы GPU.
3.  **Ловушка "System Memory Fallback":** Как только объем модели и контекста превышает доступную VRAM, драйвер задействует оперативную память. Это предотвращает краш приложения, но приводит к кратному падению скорости (в 5–10 раз), превращая интерактивный чат в фоновую задачу.
4.  **Специфика "Thinking" моделей:** Для семейства Qwen3 традиционная метрика TPS становится комплексной. Необходимо учитывать время «раздумий» (внутренней генерации цепочки рассуждений), которое может создавать задержку перед началом выдачи финального ответа.
5.  **Зависимость от питания (Hardware P-states):** На мобильных системах производительность напрямую зависит от энергопотребления. Переход GPU в низкие состояния частоты (P-states) при работе от батареи может нивелировать любые программные оптимизации.
6.  **Унификация через API:** Использование OpenAI-совместимых интерфейсов (Ollama / Llama.cpp) позволяет бесшовно переключать приложения с облачных провайдеров на локальные движки, обеспечивая полную приватность и инженерный контроль.

**Итог:** Локальный запуск моделей уровня 8B сегодня — это не просто альтернатива облакам, а высокопроизводительное и приватное решение, полностью готовое к интеграции в реальные программные продукты.